In [2]:
import matplotlib.pyplot as plt
import pandas as pd

## Plot Function 1: plotCounts

Invoked in the main Jupyter notebook. This function is designed to return counts of timestamps per activity class.

In [ ]:
def plotCounts(data: pd.DataFrame):
    fig, ax = plt.subplots()

    bars = ax.bar(
        x=data.index,
        height=data.values
    )

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_color('#DDDDDD')
    ax.tick_params(bottom=False, left=False)
    ax.set_axisbelow(True)
    ax.yaxis.grid(True, color='#EEEEEE')
    ax.xaxis.grid(False)

    ax.set_ylim([10000,25000])
    ax.get_yaxis().set_visible(False)
    bar_color = bars[0].get_facecolor()

    for bar in bars:
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 150,
            round(bar.get_height(), 1),
            horizontalalignment='center',
            color=bar_color,
            weight='bold',
            fontsize=5.5,
            rotation = 90
        )
    plt.xticks(rotation=90, size = 5.5)
    plt.title("Number of timestamps by activity")
    fig.tight_layout()

## Plot Function 2: plotStream

Invoked in the main Jupyter notebook. This function is designed to return the raw time series for both the 3-axial accelerometer and the 3-axial gyroscope, for a single workout.

In [4]:
def plotStream(data: pd.DataFrame, fig, ax, activity_map: dict):
    fig.suptitle("Accelerometer & Gyroscope Raw Time Series", fontsize = 20)
    t, acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z, activities = data.time, data.acc_x, data.acc_y, data.acc_z, data.gyr_x, data.gyr_y, data.gyr_z, data.activity
    
    # Get the min and max accelerometer and gyroscope readings across all the samples
    acc_max = max(max(acc_y), max(acc_y), max(acc_z))
    acc_min = min(min(acc_x), min(acc_x), min(acc_z))
    gyr_max = max(max(gyr_y), max(gyr_y), max(gyr_z))
    gyr_min = min(min(gyr_x), min(gyr_x), min(gyr_z))
    
    # Plot the x, y and z co-ordinate accelerometer readings
    ax[0].plot(t, acc_x, color = "red", label = "x")
    ax[0].yaxis.label.set_size(15)
    ax[0].plot(t, acc_y, color = "green", label = "y")
    ax[0].yaxis.label.set_size(15)
    ax[0].plot(t, acc_z, color = "blue", label = "z")
    ax[0].yaxis.label.set_size(15)
    
    ax[0].get_yaxis().set_visible(False)
    ax[0].legend(loc="upper right")
    
    # Plot the x, y and z co-ordinate gyroscope readings
    ax[1].plot(t, gyr_x, color = "red", label = "x")
    ax[1].yaxis.label.set_size(15)
    ax[1].plot(t, gyr_y, color = "green", label = "y")
    ax[1].yaxis.label.set_size(15)
    ax[1].plot(t, gyr_z, color = "blue", label = "z")
    ax[1].yaxis.label.set_size(15)
    
    ax[0].get_xaxis().set_visible(False)
    ax[0].get_yaxis().set_visible(False)
    ax[0].legend(loc="upper right")
    ax[1].get_xaxis().set_visible(False)
    ax[1].get_yaxis().set_visible(False)
    ax[1].legend(loc="upper right")    
    
    # Partition the workout by exercise id. Some activities will be repeated. Find the highest and lowest timestamps.
    exercises = data.groupby('exercise_id').agg(activity=('activity', 'first'), first_time=('time', 'first'), last_time= ('time', 'last'))

    # Define the colormap, generate equally spaced values between 0 and 1, then map the colours to the colourmap. Convert to hex.
    cmap = plt.cm.Blues
    num_colours = len(exercises["activity"].unique())
    values = np.linspace(0, 1, num_colours)
    colors = cmap(values)
    hex_colors = [mcolors.to_hex(color) for color in colors]
    activity_colours = {item: hex_colors[idx] for idx, item in enumerate(exercises["activity"].unique())}    
    
    for idx, row in exercises.iterrows():
        # Fill the accelerometer and gyroscope data with a colour unique to the activity
        ax[0].fill_betweenx([acc_min, acc_max], row["first_time"], row["last_time"], color=activity_colours[row["activity"]], alpha=0.4, label=activity_map[row["activity"]])
        ax[1].fill_betweenx([gyr_min, gyr_max], row["first_time"], row["last_time"], color=activity_colours[row["activity"]], alpha=0.4, label=activity_map[row["activity"]])

        # Add the activity label to the accelerometer and gyroscope plots. The label is inserted into the middle of each exercise data.
        ax[0].text(
            (row["first_time"] + row["last_time"]) / 2,
            acc_min * 0.85,
            s = activity_map[row["activity"]].replace(" ", "\n"),
            horizontalalignment='center',
            weight='bold',
            fontsize=7.5
        )
        
        
        ax[1].text(
            (row["first_time"] + row["last_time"]) / 2,
            gyr_min * 0.85,
            s = activity_map[row["activity"]].replace(" ", "\n"),
            horizontalalignment='center',
            weight='bold',
            fontsize=7.5
        )
        
    ax[0].set_title('Accelerometer Time Series')
    ax[1].set_title('Gyroscope Time Series')